In [3]:
import cv2
import numpy as np
from collections import deque
import pickle
import dill

lsFilename, outfile = ['video/190502_00-002.mp4'], 'cam00.txt'
# lsFilename, outfile = ['video/190502_01-003.mp4'], 'cam01.txt'
# lsFilename, outfile = ['video/190502_02-004.mp4'], 'cam02.txt'
# lsFilename, outfile = ['video/190502_03-005.mp4'], 'cam03.txt'
# lsFilename, outfile = ['video/190502_04-007.mp4'], 'cam04.txt'
# lsFilename, outfile = ['video/190502_05-008.mp4'], 'cam05.txt'
# lsFilename, outfile = ['video/190502_06-009.mp4'], 'cam06.txt'
# lsFilename, outfile = ['video/190502_07-010.mp4'], 'cam07.txt'
# lsFilename, outfile = ['video/190502_08-011.mp4'], 'cam08.txt'
# lsFilename, outfile = ['video/190502_09-012.mp4'], 'cam09.txt'
# lsFilename, outfile = ['video/190502_10-014.mp4'], 'cam10.txt'
# lsFilename, outfile = ['video/190502_11-015.mp4'], 'cam11.txt'

nFrame4calib = 100
bShow = True
# bShow = False
bRandomSample = True
# bRandomSample = False
# delay = 150
delay = 10
# delay = 0

nCorners = (5,7)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((nCorners[0]*nCorners[1],3), np.float32)
objp[:,:2] = np.mgrid[0:nCorners[0],0:nCorners[1]].T.reshape(-1,2) * 0.0288

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
frames = []

valid_frames = []
valid_grays = []
valid_corners = []

In [4]:
nFrame = 0
for filename in lsFilename:
    cap = cv2.VideoCapture(filename)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")

    nFrame += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print 'total :',nFrame

total : 537


In [5]:
n = 0
for filename in lsFilename:
    print filename
    cap = cv2.VideoCapture(filename)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        continue

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            find, corners = cv2.findChessboardCorners(gray, nCorners, None)
            n += 1
            print n,
            if find and (corners.reshape((-1,2)).max(0) - corners.reshape((-1,2)).min(0)).min() > 200:
                valid_frames.append(frame)
                valid_grays.append(gray)
                valid_corners.append(corners)
                print "find!", len(valid_frames),

            
        else: 
            break
    cap.release()

nFrame = len(valid_frames)
print 
print "Detected %d frames" % nFrame

filename = 'globalsave1.pkl'
dill.dump_session(filename)


video/190502_00-002.mp4
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 find! 1 81 find! 2 82 find! 3 83 find! 4 84 find! 5 85 find! 6 86 find! 7 87 find! 8 88 find! 9 89 find! 10 90 find! 11 91 find! 12 92 find! 13 93 find! 14 94 find! 15 95 find! 16 96 find! 17 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 find! 18 118 find! 19 119 find! 20 120 find! 21 121 find! 22 122 find! 23 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 find! 24 142 find! 25 143 find! 26 144 find! 27 145 find! 28 146 find! 29 147 find! 30 148 find! 31 149 find! 32 150 find! 33 151 find! 34 152 find! 35 153 find! 36 154 find! 37 155 find! 38 156 find! 39 157 find! 40 158 find! 41 159 find! 42 160 find! 43 161 find! 44 162 find! 45 163 find! 46 164 find! 47 165 find! 48 1

PicklingError: Can't pickle <type 'cv2.VideoCapture'>: it's not the same object as cv2.VideoCapture

In [4]:
# import cv2
# import numpy as np
# from collections import deque
# import pickle
# import dill

# filename = 'globalsave1.pkl'
# dill.load_session(filename)


In [6]:
delay = 150
if nFrame <= nFrame4calib:
    idxs = range(nFrame)
else:
    if bRandomSample:
        idxs = np.sort(np.random.choice(nFrame, nFrame4calib, replace=False)).tolist()
    else:
        idxs = np.linspace(0,nFrame-1,nFrame4calib).astype(int).tolist()
# idxs = range(0,len(valid_frames),3)

print len(idxs)
for idx in idxs:
    print "frame ",idx, 
    frame = valid_frames[idx]
    gray = valid_grays[idx]
    corners = valid_corners[idx]

    objpoints.append(objp)
    corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints.append(corners)
    cv2.drawChessboardCorners(frame, nCorners, corners, find)
    frames.append(frame) 
    if bShow:
        cv2.imshow('Frame',frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
print

cv2.destroyAllWindows()

cali_ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

print mtx
print dist

print len(idxs)
print len(rvecs)
print len(tvecs)

# np.savetxt(outfile, np.append(mtx, dist))

# filename = 'globalsave2.pkl'
# dill.dump_session(filename)


100
frame  2 frame  3 frame  4 frame  6 frame  8 frame  10 frame  15 frame  16 frame  18 frame  20 frame  21 frame  22 frame  23 frame  28 frame  29 frame  31 frame  34 frame  37 frame  38 frame  40 frame  42 frame  43 frame  44 frame  45 frame  46 frame  47 frame  50 frame  53 frame  55 frame  58 frame  60 frame  61 frame  62 frame  65 frame  66 frame  67 frame  69 frame  70 frame  71 frame  72 frame  74 frame  76 frame  78 frame  82 frame  85 frame  86 frame  87 frame  89 frame  91 frame  98 frame  102 frame  103 frame  104 frame  105 frame  107 frame  111 frame  112 frame  117 frame  118 frame  119 frame  120 frame  121 frame  122 frame  123 frame  124 frame  127 frame  128 frame  130 frame  131 frame  132 frame  133 frame  134 frame  141 frame  142 frame  143 frame  144 frame  145 frame  148 frame  149 frame  150 frame  153 frame  154 frame  157 frame  158 frame  160 frame  161 frame  162 frame  164 frame  166 frame  168 frame  169 frame  170 frame  171 frame  172 frame  176 frame 

In [1]:
# import cv2
# import numpy as np
# from collections import deque
# import pickle
# import dill

# filename = 'globalsave2.pkl'
# dill.load_session(filename)
# np.savetxt('out.txt', np.append(mtx, dist))

EOFError: 

In [7]:
idxs_test = range(0,len(valid_frames),10)

######################### test ##########
objpoints_test = [] # 3d point in real world space
imgpoints_test = [] # 2d points in image plane.
frames_test = []
print len(idxs_test)
for idx in idxs_test:
    print "frame ",idx
    frame = valid_frames[idx]
    gray = valid_grays[idx]
    corners = valid_corners[idx]

    objpoints_test.append(objp)
    corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints_test.append(corners)
    cv2.drawChessboardCorners(frame, nCorners, corners, find)
    frames_test.append(frame) 
    # if bShow:
    #     cv2.imshow('Frame',frame)
    #     if cv2.waitKey(delay) & 0xFF == ord('q'):
    #         break

cv2.destroyAllWindows()


cali_ret, _, _, rvecs_test, tvecs_test = cv2.calibrateCamera(objpoints_test, imgpoints_test, 
                                                             gray.shape[::-1], mtx, dist)




19
frame  0
frame  10
frame  20
frame  30
frame  40
frame  50
frame  60
frame  70
frame  80
frame  90
frame  100
frame  110
frame  120
frame  130
frame  140
frame  150
frame  160
frame  170
frame  180


In [22]:
bShow = True
delay = 500


tot_error = 0
tot_error_undist = 0
mean_error = 0


def MyProjection(objp, rvec, tvec, mtx, dst):
    rot, _ = cv2.Rodrigues(rvec)
    imgp = np.dot(rot,objp.T) + tvec
    imgp = imgp / imgp[2,:]

    x = imgp[0,:]
    y = imgp[1,:]
    
    r = np.sqrt(x**2 + y**2)

    k1, k2, p1, p2, k3 = dst
    c = (1 + k1*r**2 + k2*r**4 + k3*r**6)/(1)
    x_new = c*x + 2*p1*x*y + p2*(r**2 + 2*x**2)
    y_new = c*y + p1*(r**2 + 2*y**2) + 2*p2*x*y

    imgp[0,:]= x_new
    imgp[1,:]= y_new
    

    imgp = np.dot(mtx,imgp)[:2,:].T.astype(objp.dtype)
    return imgp


# for frame, rvec, tvec, corners, objp in zip(frames_test, rvecs_test, tvecs_test, imgpoints_test, objpoints_test):
for i in range(len(frames_test)):
    frame, rvec, tvec, corners, objp = frames_test[i], rvecs_test[i], tvecs_test[i], imgpoints_test[i], objpoints_test[i]
    imgp1 = MyProjection(objp, rvec, tvec, mtx, dist[0])
    # imgpoints1, _ = cv2.projectPoints(objp, rvec, tvec, mtx, dist)
    # print cv2.norm(imgpoints1.reshape(imgp1.shape), imgp1, cv2.NORM_L2)
    
    imgpoints2, _ = cv2.projectPoints(objp, rvec, tvec, mtx, dist)
    print imgp1
    print imgpoints2.reshape(-1,2)
    break
    for p in imgp1:
        cv2.circle(frame, tuple(p.astype(int)), 1, (0,0,255), 3)

    tot_error += cv2.norm(corners,imgp1.reshape(corners.shape), cv2.NORM_L2)/len(imgp1)

    if bShow:
        cv2.imshow('Frame',frame)
        # cv2.imshow('Dst',dst)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
           break

print "total error: ", tot_error/len(objpoints)








[[ 647.10620117  436.60855103]
 [ 650.69250488  487.58682251]
 [ 654.31011963  538.21411133]
 [ 657.95874023  588.50750732]
 [ 661.63696289  638.48358154]
 [ 595.53137207  441.22897339]
 [ 599.36584473  492.56393433]
 [ 603.22375488  543.54309082]
 [ 607.10522461  594.18518066]
 [ 611.00909424  644.50836182]
 [ 543.22998047  445.98977661]
 [ 547.32128906  497.69338989]
 [ 551.42608643  549.03582764]
 [ 555.54534912  600.0380249 ]
 [ 559.67797852  650.72021484]
 [ 490.15267944  450.89608765]
 [ 494.51080322  502.98251343]
 [ 498.87017822  554.70196533]
 [ 503.23254395  606.07794189]
 [ 507.59738159  657.13305664]
 [ 436.23779297  455.95355225]
 [ 440.87423706  508.4397583 ]
 [ 445.49697876  560.5526123 ]
 [ 450.10870361  612.31860352]
 [ 454.70974731  663.76306152]
 [ 381.4100647   461.16900635]
 [ 386.33792114  514.07489014]
 [ 391.23431396  566.60046387]
 [ 396.10308838  618.77545166]
 [ 400.94561768  670.62811279]
 [ 325.58218384  466.55072021]
 [ 330.81588745  519.89910889]
 [ 335.9

In [23]:
tot_error = 0
for i in range(len(frames_test)):
    frame, rvec, tvec, corners, objp = frames_test[i], rvecs_test[i], tvecs_test[i], imgpoints_test[i], objpoints_test[i]

    imgpoints2, _ = cv2.projectPoints(objp, rvec, tvec, mtx, dist)
    error = cv2.norm(imgpoints_test[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

    for p in imgpoints2:
        cv2.circle(frame, tuple(p[0].astype(int)), 1, (0,255,255), 3)

    if bShow:
        cv2.imshow('Frame',frame)
        # cv2.imshow('Dst',dst)
        if cv2.waitKey(150) & 0xFF == ord('q'):
           break


print "total error: ", tot_error/len(objpoints)

total error:  0.0293371690765
